In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from google.colab import drive
import ee
import geemap.foliumap as geemap

In [2]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import explore

path = "/content/drive/MyDrive/CAFO_data/Misc_global/shapefiles/"

Mounted at /content/drive


In [3]:
# TURKEY - region boundary coords defined by hand using Google Maps

turkey = {"Name": "Turkey", "Code": "TUR"}
turkey["CRS"] = "EPSG:5636"
turkey["Boundary latlon"] = [(38.675596, 30.745101), (38.693221, 30.607857),\
                             (38.788433, 30.603912), (38.810095, 30.636428),\
                             (38.821603, 30.691116), (38.722434, 30.699930)]
turkey["Boundary lonlat"] = [x[::-1] for x in turkey["Boundary latlon"]]
turkey["Boundary"]= Polygon(turkey["Boundary lonlat"])

# Save the region boundary for later use
gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=[turkey["Boundary"]])
turkey["Boundary gdf"] = gdf
gdf.to_file(f"{path}Turkey.shp")

In [4]:
# INDIA (Tamil Nadu)

india = {"Name": "India", "Code": "IND"}
india["CRS"] = "EPSG:7785"

india["Boundary latlon"] = [(11.264466, 78.195514), (11.257338, 78.132436),\
                            (11.278384, 78.094542), (11.359775, 78.122547),\
                            (11.351684, 78.183257), (11.282473, 78.202641)]
india["Boundary lonlat"] = [x[::-1] for x in india["Boundary latlon"]]
india["Boundary"]= Polygon(india["Boundary lonlat"])

# Save the region boundary for later use
gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=[india["Boundary"]])
india["Boundary gdf"] = gdf
gdf.to_file(f"{path}India.shp")

In [7]:
# THAILAND

thailand = {"Name": "Thailand", "Code": "THA"}
thailand["CRS"] = "EPSG:24047"

thailand["Boundary latlon"] = [(13.274320, 99.816416), (13.262629, 99.723449),\
                               (13.473833, 99.591679), (13.489666, 99.724536)]
thailand["Boundary lonlat"] = [x[::-1] for x in thailand["Boundary latlon"]]
thailand["Boundary"]= Polygon(thailand["Boundary lonlat"])

# Save the region boundary for later use
gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=[thailand["Boundary"]])
thailand["Boundary gdf"] = gdf
gdf.to_file(f"{path}Thailand.shp")

In [8]:
min_building_size=800

where = thailand

In [9]:
# Obtain a feature collection of buildings > min_building_size within the
# specified boundary

def get_buildings(area, min_building_size, code):
  gdf = gpd.GeoDataFrame(crs="EPSG:4326", geometry=[area["Boundary"]])
  geom = geemap.geopandas_to_ee(gdf[['geometry']])

  buildings_fc = (
      ee.FeatureCollection(f"projects/sat-io/open-datasets/VIDA_COMBINED/{code}")
      .filter(ee.Filter.gt('area_in_meters', min_building_size))
      .filterBounds(geom)
  )

  return buildings_fc, geom

In [10]:
# Combine closely-spaced buildings into a single polygon, and define a box
# around the centroid of each clump. The box should be roughly the same size as
# the Sentinel snippets I eventually use in model training and application;
# they don't get used directly but are useful for visualizing what we're
# working with

def merge_and_make_box(area, buildings_fc):
  buildings = geemap.ee_to_gdf(buildings_fc)

  merged = buildings.to_crs(area["CRS"]).buffer(50).union_all()
  merged = gpd.GeoDataFrame(merged.geoms).set_geometry(0).set_crs(area["CRS"])

  boxes = gpd.GeoDataFrame(merged.centroid.buffer(200).envelope)
  boxes = boxes.rename(columns={0: "geometry"}).set_geometry("geometry").to_crs("EPSG:4326")

  # remove boxes that intersect with boundary edges, as they'd be discarded
  # when the Sentinel images are created, anyway

  boxes = boxes.sjoin(area["Boundary gdf"], how="inner", predicate="within")
  boxes = boxes.drop(columns=["index_right"]).reset_index(drop=True)

  print(f"Went from {len(buildings)} buildings to {len(boxes)} boxes")

  return merged, boxes

In [11]:
buildings_fc, boundary = get_buildings(where, min_building_size, where["Code"])
merged, boxes = merge_and_make_box(where, buildings_fc)

Went from 1923 buildings to 254 boxes


In [12]:
# Visualize the entire area, including its boundary, all the large buildings,
# the merged building polygons, and the boxes

boxes_fc = geemap.geopandas_to_ee(boxes)
merged_fc = geemap.geopandas_to_ee(merged.to_crs("EPSG:4326"))

os.environ["HYBRID"] = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

boundary_viz = {
  'color': 'red',
  'width': 2,
  'fillColor': '00000000'
}

buildings_viz = {
  'color': 'yellow',
  'width': 2,
  'fillColor': '00000000'
}

merged_viz = {
  'color': 'cyan',
  'width': 2,
  'fillColor': '00000000'
}

boxes_viz = {
  'color': 'blue',
  'width': 2,
  'fillColor': '00000000'
}

Map = geemap.Map()
Map.centerObject(buildings_fc.first().geometry(), 13)
Map.add_basemap("HYBRID")
Map.addLayer(boundary.style(**boundary_viz), {}, "Boundary")
Map.addLayer(buildings_fc.style(**buildings_viz), {}, "Buildings")
Map.addLayer(merged_fc.style(**merged_viz), {}, "Merged")
Map.addLayer(boxes_fc.style(**boxes_viz), {}, "Boxes")

Map

In [18]:
# Step through each of the boxes. Use the "reject" option to mark the CAFOs,
# we'll use the returned list to label them afterwards

#cafos1 = explore.loop_over_buildings(boxes[:50])
#cafos2 = explore.loop_over_buildings(boxes[50:100])
cafos3 = explore.loop_over_buildings(boxes[100:])

Output hidden; open in https://colab.research.google.com to view.

In [19]:
print(cafos1)
print(cafos2)
print(cafos3)
cafos = cafos1 + cafos2 + cafos3

[0, 1, 3, 6, 12, 13, 14, 20, 21, 22, 23, 29, 30, 34, 38, 39, 42, 44, 46, 47]
[50, 51, 54, 56, 58, 60, 71, 74, 78, 81, 84, 85, 86, 87, 88]
[102, 105, 106, 107, 117, 118, 121, 123, 126, 131, 146, 147, 148, 149, 153, 154, 155, 157, 165, 166, 167, 168, 169, 175, 176, 177, 182, 183, 184, 188, 189, 197, 198, 199, 214, 215, 223, 224, 225, 226, 229, 251, 252]


In [20]:
# Create a df where each row contains the centroid of one of the above boxes,
# is labelled as Unknown CAFO or Non-farm, and has the same columns as all the
# training datasets (Mexico, Iowa, etc.)

# This df doesn't identify any particular building as the main or central one
# in any of the boxes, or contain info about building sizes etc. It could be
# made to, but for now it does not.

def make_final_df(area, boxes, cafos):
  candidates = boxes.copy()

  centroids = candidates.to_crs(area["CRS"]).geometry.centroid.to_crs("EPSG:4326")
  candidates.loc[:, "geometry"] = centroids

  candidates.loc[candidates.index.isin(cafos), 'Farm type'] = "Unknown CAFO"
  candidates.loc[~candidates.index.isin(cafos), 'Farm type'] = "Non-farm"
  coldict = {"Dataset name": area["Name"], "Parent coords": None,\
             "Number of animals": np.nan, "Area (sq m)": np.nan,\
             "Length (m)": np.nan, "Aspect ratio": np.nan}
  for col, val in coldict.items():
    candidates.loc[:, col] = val

  candidates = explore.re_order(candidates)
  print(f'Saving {len(candidates[candidates["Farm type"] == "Unknown CAFO"])} farm coords')
  print(f'Saving {len(candidates[candidates["Farm type"] == "Non-farm"])} non-farm coords')

  path = "/content/drive/MyDrive/CAFO_data/forTraining/interim_files/"
  candidates.to_pickle(f'{path}{area["Name"]}_bldgs.pkl')

In [21]:
make_final_df(where, boxes, cafos)

Saving 78 farm coords
Saving 176 non-farm coords
